# QU CBE Introduction to Python Workshop - Introduction to Econometrics in Python

***Mohammed Ait Lahcen, Department of Finance and Economics, College of Business and Economics, Qatar University***

There are many libraries in Python which cover statistics and econometrics, for example:
- `statsmodels`
- `scikit_learn`
- `linearmodels`
- `PyMC3`

In this tutorial we take a look at `statsmodels` and `linearmodels`.

In [1]:
# We start by importing pandas
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

### Loading data

We are going to use Our World in Data covid database available in the same folder.

In [2]:
data = pd.read_csv('owid-covid-data.csv', sep=',',header=0)

Subnational locations (England, Northern Ireland, Scotland, Wales, Northern Cyprus…) and international aggregates (World, continents, European Union…) can be identified by their isocode that starts with OWID. We want to drop these entities from our data:

In [3]:
data = data[~data['iso_code'].str.startswith('OWID_')]

We can also change the column date from string to datetime data type to make sure pandas recognizes its content as dates:

In [4]:
data.loc[:,'date'] = pd.to_datetime(data.loc[:,'date'])

In [5]:
data = data.set_index(['iso_code','date'],verify_integrity=True) # verify that the multindex is unique

In [6]:
data.sort_index(level=['iso_code','date'], ascending=[1, 1], inplace=True)

In [7]:
# create a new column
data['case_fatality_rate'] = data['total_deaths_per_million'] / data['total_cases_per_million']

## OLS regression

We will use statsmodels' OLS (ordinary least squares) function to estimate our linear regression model:

In [8]:
y = data['total_cases_per_million']
x = sm.add_constant(data['stringency_index'])
pooled_ols = sm.OLS(y,x,missing='drop')

We then use .fit() to estimate the parameters

In [9]:
results = pooled_ols.fit()
results

In [10]:
print(results.summary())

                               OLS Regression Results                              
Dep. Variable:     total_cases_per_million   R-squared:                       0.000
Model:                                 OLS   Adj. R-squared:                 -0.000
Method:                      Least Squares   F-statistic:                 1.812e-05
Date:                     Wed, 27 Oct 2021   Prob (F-statistic):              0.997
Time:                             11:51:20   Log-Likelihood:            -1.2052e+06
No. Observations:                   102494   AIC:                         2.410e+06
Df Residuals:                       102492   BIC:                         2.411e+06
Df Model:                                1                                         
Covariance Type:                 nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------

Now we can try to control for the number of tests:

In [11]:
y = data['total_cases_per_million']
x = sm.add_constant(data[['stringency_index','total_tests_per_thousand','gdp_per_capita','aged_65_older','diabetes_prevalence','hospital_beds_per_thousand']])
pooled_ols = sm.OLS(y,x,missing='drop').fit(cov_type='HC3')
print(pooled_ols.summary())

                               OLS Regression Results                              
Dep. Variable:     total_cases_per_million   R-squared:                       0.326
Model:                                 OLS   Adj. R-squared:                  0.326
Method:                      Least Squares   F-statistic:                     2053.
Date:                     Wed, 27 Oct 2021   Prob (F-statistic):               0.00
Time:                             11:51:20   Log-Likelihood:            -5.5083e+05
No. Observations:                    47524   AIC:                         1.102e+06
Df Residuals:                        47517   BIC:                         1.102e+06
Df Model:                                6                                         
Covariance Type:                       HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------

We redo the same regression using total deaths per million instead:

In [12]:
y = data['total_deaths_per_million']
x = sm.add_constant(data[['stringency_index','total_tests_per_thousand','gdp_per_capita','aged_65_older','diabetes_prevalence','hospital_beds_per_thousand']])
pooled_ols = sm.OLS(y,x,missing='drop').fit(cov_type='HC3')
print(pooled_ols.summary())

                               OLS Regression Results                               
Dep. Variable:     total_deaths_per_million   R-squared:                       0.111
Model:                                  OLS   Adj. R-squared:                  0.111
Method:                       Least Squares   F-statistic:                     997.6
Date:                      Wed, 27 Oct 2021   Prob (F-statistic):               0.00
Time:                              11:51:20   Log-Likelihood:            -3.6654e+05
No. Observations:                     46129   AIC:                         7.331e+05
Df Residuals:                         46122   BIC:                         7.332e+05
Df Model:                                 6                                         
Covariance Type:                        HC3                                         
                                 coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------

## Panel regression

I'm going to try the same regression by adding country fixed effects. `PanelOLS` exploits the index structure we defined earlier so we don't need to specifiy which variables are entities and time dates.

In [13]:
y = data['total_cases_per_million']
x = sm.add_constant(data[['stringency_index','total_tests_per_thousand','gdp_per_capita','aged_65_older']])
model = PanelOLS(y, x, time_effects=0, entity_effects=1,drop_absorbed=True)
results = model.fit(cov_type='clustered',cluster_entity=True)
print(results)

C:\Users\MA19310\Anaconda3\envs\_updated_env\lib\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
C:\Users\MA19310\Anaconda3\envs\_updated_env\lib\site-packages\linearmodels\panel\model.py:1796: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

gdp_per_capita, aged_65_older

  warnings.warn(


                             PanelOLS Estimation Summary                             
Dep. Variable:     total_cases_per_million   R-squared:                        0.3694
Estimator:                        PanelOLS   R-squared (Between):              0.1812
No. Observations:                    50161   R-squared (Within):               0.3694
Date:                     Wed, Oct 27 2021   R-squared (Overall):              0.2928
Time:                             11:51:20   Log-likelihood                -5.674e+05
Cov. Estimator:                  Clustered                                           
                                             F-statistic:                   1.466e+04
Entities:                              119   P-value                           0.0000
Avg Obs:                            421.52   Distribution:                 F(2,50040)
Min Obs:                            1.0000                                           
Max Obs:                            633.00   F-statist

Notice that to make `PanelOLS` automatically drop the absorbed variables we need to add `drop_absorbed=True`